Cek jumlah beserta struktur datasets

In [4]:
import os

root_path = "D:\Pothole Vision - AI Road Damage Detection\dataset\RDD2022_all_countries"
def count_files(path):
    return len([f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]) if os.path.exists(path) else 0

summary = []

for country in os.listdir(root_path):
    country_path = os.path.join(root_path, country)
    if os.path.isdir(country_path):
        train_images = count_files(os.path.join(country_path, "train", "images"))
        test_images = count_files(os.path.join(country_path, "test", "images"))
        xml_files = count_files(os.path.join(country_path, "train", "annotations", "xmls"))
        summary.append(f"{country} -> Train Images: {train_images}, Test Images: {test_images}, Annotations: {xml_files}")

for line in summary:
    print(line)

China_Drone -> Train Images: 2401, Test Images: 0, Annotations: 2401
China_MotorBike -> Train Images: 1977, Test Images: 500, Annotations: 1977
Czech -> Train Images: 2829, Test Images: 709, Annotations: 2829
India -> Train Images: 7706, Test Images: 1959, Annotations: 7706
Japan -> Train Images: 10506, Test Images: 2627, Annotations: 10506
Norway -> Train Images: 8161, Test Images: 2040, Annotations: 8161
United_States -> Train Images: 4805, Test Images: 1200, Annotations: 4805


China_drone tidak punya folder test, abaikan?
Selanjutnya Saya akan split datset dari Train & Test menjadi Train, Test, & Val


In [ ]:
Jika dataset DIGABUNG semua negara menjadi satu dataset besar:
Keuntungan:

Model akan lebih general karena belajar dari berbagai jenis jalan, cuaca, kamera.

Bisa membantu jika nanti digunakan di Indonesia yang belum punya data.

Jumlah data menjadi sangat besar (10.000++), sangat bagus untuk deep learning.

Kekurangan:

Bisa menyebabkan bias ke negara dengan data terbanyak (misalnya India, Japan).

Anotasi antar negara mungkin memiliki inkonsistensi kecil (labeling style, noise).

In [ ]:
Rekomendasi untuk kasus ini:
Karena kamu akan pakai untuk Indonesia, tapi belum punya data lokal, maka:

Gabungkan semua negara → latih model global, supaya kuat terhadap variasi.

Simpan metadata negara asalnya → bisa dipakai untuk evaluasi per negara.

Nanti, jika ada data Indonesia, kamu bisa fine-tune model global ke data lokal.

In [1]:
import os
import random
import shutil
import xml.etree.ElementTree as ET
import pandas as pd

# Lokasi dataset dan output
root_path = "D:\\Pothole Vision - AI Road Damage Detection\\dataset\\RDD2022_all_countries"
output_path = "D:\\Pothole Vision - AI Road Damage Detection\\dataset-mix"
train_val_split = 0.8  # 80% untuk train, 20% untuk val

# Membuat direktori output
os.makedirs(os.path.join(output_path, 'dataset-mix', 'train', 'images'), exist_ok=True)
os.makedirs(os.path.join(output_path, 'dataset-mix', 'train', 'annotations'), exist_ok=True)
os.makedirs(os.path.join(output_path, 'dataset-mix', 'val', 'images'), exist_ok=True)
os.makedirs(os.path.join(output_path, 'dataset-mix', 'val', 'annotations'), exist_ok=True)
os.makedirs(os.path.join(output_path, 'dataset-mix', 'test', 'images'), exist_ok=True)

# Metadata untuk csv
metadata = []

# Fungsi untuk meng-copy file gambar dan anotasi ke folder baru
def copy_file(src_file, dest_dir):
    shutil.copy(src_file, dest_dir)

# Proses pemindahan file per negara
for country in os.listdir(root_path):
    country_path = os.path.join(root_path, country)
    
    if os.path.isdir(country_path):  # Mengecek apakah ini folder negara
        # Folder gambar dan anotasi
        images_path = os.path.join(country_path, 'train', 'images')
        annotations_path = os.path.join(country_path, 'train', 'annotations', 'xmls')
        
        # Ambil daftar gambar dan anotasi
        image_files = [f for f in os.listdir(images_path) if f.endswith('.jpg')]
        annotation_files = [f for f in os.listdir(annotations_path) if f.endswith('.xml')]

        # Gabungkan data gambar dan anotasi ke dalam dataset-mix
        for image, annotation in zip(image_files, annotation_files):
            image_path = os.path.join(images_path, image)
            annotation_path = os.path.join(annotations_path, annotation)
            
            # Tentukan split (train/val)
            if random.random() < train_val_split:
                split = 'train'
                dest_image = os.path.join(output_path, 'dataset-mix', 'train', 'images', image)
                dest_annotation = os.path.join(output_path, 'dataset-mix', 'train', 'annotations', annotation)
            else:
                split = 'val'
                dest_image = os.path.join(output_path, 'dataset-mix', 'val', 'images', image)
                dest_annotation = os.path.join(output_path, 'dataset-mix', 'val', 'annotations', annotation)

            # Copy gambar dan anotasi
            copy_file(image_path, os.path.dirname(dest_image))
            copy_file(annotation_path, os.path.dirname(dest_annotation))
            
            # Menambahkan metadata
            metadata.append({'filename': image, 'country': country, 'split': split})

# Menyimpan metadata ke dalam CSV
metadata_df = pd.DataFrame(metadata)
metadata_df.to_csv(os.path.join(output_path, 'dataset-mix', 'metadata.csv'), index=False)

print("Dataset telah berhasil diproses dan disalin ke folder dataset-mix.")


Dataset telah berhasil diproses dan disalin ke folder dataset-mix.
